In [37]:
import json
import numpy as np
import torch
import scipy.sparse as sparse

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

In [55]:
print(dataset[3])

Data(edge_index=[2, 16], edge_attr=[16, 3], x=[8, 9], y=[1, 1], num_nodes=8)


In [86]:
from tqdm import tqdm
node_list = []
labels = []
edges = []
edge_feats = []
node_feats = []
num_nodes = 0

In [87]:
for g in tqdm(dataset):
    node_list.append(np.arange(g["num_nodes"]) + num_nodes)  # All the nodes are considered in a single graph
    edges.append(np.stack(g["edge_index"]).T + num_nodes)
    edge_feats.append(g["edge_attr"])
    node_feats.append(g["x"])
    num_nodes += g["num_nodes"]

100%|██████████| 642/642 [00:00<00:00, 13341.64it/s]


In [54]:
_edges = np.concatenate(edges, axis=0)
_edge_feats = np.concatenate(edge_feats)
_node_feats = np.concatenate(node_feats)

In [89]:
_node_list = sparse.lil_matrix((len(dataset), num_nodes))
for i, indices in enumerate(node_list):
    _node_list[i, indices] = 1

In [91]:
data = {
    "node_feats": node_feats,
    "edge": _edges,
    "edge_feats": _edge_feats,
    "node_list": _node_list,
}
np.savez_compressed("ogbg-molfreesolv.npz", **data)

/Users/tuyiwen/Library/Python/3.8/lib/python/site-packages/numpy/lib/npyio.py:696: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  val = np.asanyarray(val)
/Users/tuyiwen/Library/Python/3.8/lib/python/site-packages/numpy/lib/npyio.py:696: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


In [36]:
task_data={
    "train" :train_idx,
    "val" :valid_idx,
    "test" :test_idx
}
np.savez_compressed("ogbg-molfreesolv_task.npz",**task_data)